In [1]:
import netket as nk
import netket.experimental as nkx
import netket.nn as nknn

from math import pi

from netket.experimental.operator.fermion import destroy as c
from netket.experimental.operator.fermion import create as cdag
from netket.experimental.operator.fermion import number as nc

from vmc_torch.fermion_utils import generate_random_fpeps, product_bra_state
import quimb.tensor as qtn
from autoray import do
import symmray as sr
import pickle
import os
os.environ['NUMBA_NUM_THREADS'] = '20'

# Define the lattice shape
L = 4  # Side of the square
Lx = int(L/2)
Ly = int(L/2)
# graph = nk.graph.Square(L)
graph = nk.graph.Grid([Lx,Ly], pbc=False)
N = graph.n_nodes


# Define the fermion filling and the Hilbert space
N_f = int(Lx*Ly/2)
hi = nkx.hilbert.SpinOrbitalFermions(N, s=1/2, n_fermions=N_f)
# hi = nkx.hilbert.SpinOrbitalFermions(N, s=1/2, n_fermions_per_spin=(N_f//2,N_f//2))


# Define the Hubbard Hamiltonian
t = 1.0
U = 8.0
mu = 0.0

H = 0.0
for (i, j) in graph.edges(): # Definition of the Hubbard Hamiltonian
    for spin in (1,-1):
        H -= t * (cdag(hi,i,spin) * c(hi,j,spin) + cdag(hi,j,spin) * c(hi,i,spin))
for i in graph.nodes():
    H += U * nc(hi,i,+1) * nc(hi,i,-1)


# Exact diagonalization of the Hamiltonian for benchmark
sp_h = H.to_sparse() # Convert the Hamiltonian to a sparse matrix
from scipy.sparse.linalg import eigsh
from scipy.linalg import eigh
try:
    eig_vals, eig_vecs = eigsh(sp_h, k=2, which="SA")
    E_gs = eig_vals[0]
    psi_gs = eig_vecs[:,0]
except:
    eig_val, eig_vec = eigh(sp_h.toarray())
    E_gs = eig_val
    psi_gs = eig_vec[:,0]

print("Exact ground state energy per site:", E_gs/N)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
/home/sijingdu/anaconda3/envs/vmc_torch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sijingdu/anaconda3/envs/vmc_torch/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/sijingdu/anaconda3/envs/vmc_torch/lib/python3.9/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


Exact ground state energy per site: -0.8019377358048383


In [35]:
import jax
import numpy as np
random_seed = np.random.randint(2**32 - 1)
config = hi.random_state(key=jax.random.PRNGKey(random_seed))

# --- Utils ---
def from_netket_config_to_quimb_config(netket_configs):
    def func(netket_config):
        """Translate netket spin-1/2 config to tensor network product state config"""
        total_sites = len(netket_config)//2
        spin_up = netket_config[:total_sites]
        spin_down = netket_config[total_sites:]
        sum_spin = spin_up + spin_down
        quimb_config = np.zeros(total_sites, dtype=int)
        for i in range(total_sites):
            if sum_spin[i] == 0:
                quimb_config[i] = 0
            if sum_spin[i] == 2:
                quimb_config[i] = 3
            if sum_spin[i] == 1:
                if spin_down[i] == 1:
                    quimb_config[i] = 1
                else:
                    quimb_config[i] = 2
        return quimb_config
    if len(netket_configs.shape) == 1:
        return func(netket_configs)
    else:
        # batched
        return np.array([func(netket_config) for netket_config in netket_configs])

def from_quimb_config_to_netket_config(quimb_config):
    """Translate tensor network product state config to netket spin-1/2 config"""
    total_sites = len(quimb_config)
    spin_up = np.zeros(total_sites, dtype=int)
    spin_down = np.zeros(total_sites, dtype=int)
    for i in range(total_sites):
        if quimb_config[i] == 0:
            spin_up[i] = 0
            spin_down[i] = 0
        if quimb_config[i] == 1:
            spin_up[i] = 0
            spin_down[i] = 1
        if quimb_config[i] == 2:
            spin_up[i] = 1
            spin_down[i] = 0
        if quimb_config[i] == 3:
            spin_up[i] = 1
            spin_down[i] = 1
    return np.concatenate((spin_up, spin_down))


# Spinful PEPS

In [3]:
D = 4
seed = 2
symmetry = 'U1'
spinless = False
peps = generate_random_fpeps(Lx, Ly, D=4, seed=2, symmetry=symmetry, Nf=N_f, spinless=spinless)[0]
edges = qtn.edges_2d_square(Lx, Ly, cyclic=False)
site_info = sr.utils.parse_edges_to_site_info(
    edges,
    D,
    phys_dim=4,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
)

t = 1.0
U = 8.0
mu = 0.0

terms = {
    (sitea, siteb): sr.fermi_hubbard_local_array(
        t=t, U=U, mu=mu,
        symmetry=symmetry,
        coordinations=(
            site_info[sitea]['coordination'],
            site_info[siteb]['coordination'],
        ),
    ).fuse((0, 1), (2, 3))
    for (sitea, siteb) in peps.gen_bond_coos()
}
ham = qtn.LocalHam2D(Lx, Ly, terms)

su = qtn.SimpleUpdateGen(peps, ham, compute_energy_per_site=True,D=D, compute_energy_opts={"max_distance":2}, gate_opts={'cutoff':1e-10})
su.evolve(20, tau=0.3)
peps = su.state
gs = su.get_state()
# peps.arrays

n=20, tau=0.3000, energy~-0.765082: 100%|##########| 20/20 [00:00<00:00, 54.98it/s]


In [36]:
N_terms = {
    site: sr.fermi_number_operator_spinful_local_array(
        symmetry=symmetry
    )
    for site in peps.gen_site_coos()
}
gs = su.get_state()
gs.compute_local_expectation(
    N_terms, normalized=True, max_bond=64,
)/N, gs.compute_local_expectation(
    ham.terms, normalized=True, max_bond=128,
)/N,

(np.float64(0.5), -0.7650824957671803)

In [38]:
# Netket Hamiltonian energy check
import numpy as np
all_configs = hi.all_states()
if not spinless:
    # all_configs = [from_netket_config_to_quimb_config(config) for config in all_configs]
    all_configs = from_netket_config_to_quimb_config(all_configs)

psi_vec = np.array([gs.get_amp(config).contract() for config in all_configs])

hamiltonian = H.to_dense()
psi_vec = psi_vec/np.linalg.norm(psi_vec)
print((psi_vec.conj().T @ (hamiltonian @ psi_vec))/N)

-0.7650824957671805


In [42]:
config_0 = all_configs[20]
netket_config_0 = from_quimb_config_to_netket_config(config_0)
netket_config_0, config_0

(array([0, 1, 1, 0, 0, 0, 0, 0]), array([0, 2, 2, 0]))

In [28]:
H.get_conn(netket_config_0)

(array([[1, 0, 1, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0, 0]]),
 array([-1.,  1., -1.,  1.]))

In [35]:
# Quimb energy check
gs.compute_local_expectation(
    ham.terms, normalized=True, max_bond=64,
)/N

-0.7665835745817513

In [36]:
def to_array_check(peps):
    all_config = hi.all_states()
    if not spinless:
        all_config = [from_netket_config_to_quimb_config(config) for config in all_config]
    psi = np.asarray([peps.get_amp(config, conj=True).contract() for config in all_config])
    # print(psi)
    psi = psi/do('linalg.norm', psi)
    return psi

def compute_energy_check(model, hamiltonian):
    psi_gs = to_array_check(model)
    return psi_gs.conj().T@(hamiltonian@psi_gs)/N

hamiltonian = H

compute_energy_check(peps, hamiltonian)

np.float64(-0.7665835745817514)

In [5]:
n_site = N_terms[(0,0)]
inda, indb = 'a', 'b'
product_state1 = sr.FermionicArray.from_blocks(blocks={(2,):np.array([1])}, duals=(True,),symmetry='U1', charge=2, oddpos=1)
product_state2 = sr.FermionicArray.from_blocks(blocks={(2,):np.array([1])}, duals=(True,),symmetry='U1', charge=2, oddpos=2)
n_site_tensor = qtn.Tensor(data=n_site, inds=(inda,indb), tags=(f'I{inda},{indb}',))
product_state_tensor1 = qtn.Tensor(data=product_state1, inds=(inda,), tags=(f'I{inda}',))
product_state_tensor2 = qtn.Tensor(data=product_state2, inds=(indb,), tags=(f'I{indb}',))
(product_state_tensor1.conj()|n_site_tensor|product_state_tensor2).contract(), (product_state_tensor1|product_state_tensor1.conj()).contract(), (n_site_tensor|product_state_tensor1.conj()).contract().data.to_dense()

(2.0, 1, array([0., 0., 0., 2.]))

# Cyclic PEPS

In [5]:
peps = qtn.PEPS.rand(4, 4, 4, cyclic=True)

In [13]:
# SU in quimb
D = 4
seed = 2
symmetry = 'Z2'
cyclic = True
# peps, parity_config = generate_random_fpeps(Lx, Ly, D, seed, symmetry, Nf=N_f)

edges = qtn.edges_2d_square(Lx, Ly, cyclic=cyclic)
site_info = sr.utils.parse_edges_to_site_info(
    edges,
    D,
    phys_dim=2,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
)

peps = qtn.TensorNetwork()
parity_config = np.zeros(Lx*Ly, dtype=int)
rng = np.random.default_rng(seed)
for site, info in sorted(site_info.items()):
    tid = site[0] * Ly + site[1]
    # bond index charge distribution
    block_indices = [
        sr.BlockIndex({0: d // 2, 1: d // 2}, dual=dual)
        for d, dual in zip(info["shape"][:-1], info["duals"][:-1])
    ]

    # physical index
    p = info['shape'][-1]
    block_indices.append(
        sr.BlockIndex({0: p // 2, 1: p // 2}, dual=info["duals"][-1])
    )

    data = sr.Z2FermionicArray.random(
        block_indices,
        charge=1 if parity_config[tid] else 0,
        seed=rng,
        oddpos=2*tid,
    )

    peps |= qtn.Tensor(
        data=data,
        inds=info["inds"],
        tags=info["tags"],
    )

# required to view general TN as an actual PEPS
for i, j in site_info:
    peps[f"I{i},{j}"].add_tag([f"X{i}", f"Y{j}"])

peps.view_as_(
    fPEPS,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
    x_tag_id="X{}",
    y_tag_id="Y{}",
    Lx=Lx,
    Ly=Ly,
)
peps = peps.copy() # set symmetry during initialization

In [ ]:
peps.draw()